# ACM ICAIF-25 AI Agentic Retrieval Grand Challenge – Baseline

This notebook provides a **complete baseline solution** for the ranking evaluation tasks using **Databricks GPT OSS 120B** with a financial analyst system prompt.

**Note**: this notebook works directly on Kaggle paths. To work locally, we need to resolve paths with `config.py`.

---

## Overview

The competition consists of two main ranking tasks:

1. **Document Ranking** – Identify and rank the five most relevant documents.  
2. **Chunk Ranking** – Identify and rank the five most relevant text chunks.

---

## Processing Pipeline

- **Data Loading**  
  Reads evaluation data from JSONL files.

- **Token Analysis**  
  Checks input size to decide whether it exceed the context length of the model..

- **Smart Ranking**  
  - *Normal cases*: Single-stage ranking.  
  - *High-token cases*: Multi-stage divide-and-conquer ranking to handle large inputs efficiently.

- **Result Compilation**  
  Combines model outputs and prepares a final CSV submission file.

---

## Output

- **kaggle_submission.csv**  
  Ready-to-submit file containing the required `sample_id` and `target_index` columns.

- **Comprehensive Statistics**  
  Summarized metrics and analysis of ranking results.

- **Top-5 Rankings**  
  Returns the five most relevant items for each query as required by the challenge.

---

## Usage

This notebook enables **end-to-end evaluation**: from loading data to generating a Kaggle-ready submission file.  
Simply run the pipeline and upload the generated `kaggle_submission.csv` to the competition platform.

## Setup & Imports

In [1]:
# !pip install openai tiktoken python-dotenv pydantic tqdm

In [1]:
import asyncio
import csv
import json
import os
import traceback
from typing import Dict, List

import tiktoken
from dotenv import load_dotenv
from openai import AsyncOpenAI
from pydantic import BaseModel
from tqdm.asyncio import tqdm

load_dotenv()

True

## Configuration and Model Setup
Search for “Databricks free trial” as shown in the screenshot and sign up.

1. Use Model Serving to set up a Databricks ***model endpoint***,
2. Prepare an access token from the ***Settings page***.

In [2]:
# Environment variables
DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Check if environment variables are set
if not DATABRICKS_TOKEN:
    print("⚠️ DATABRICKS_TOKEN not found in environment variables")
    print("Please set your Databricks token in your environment or .env file")
if not OPENAI_API_KEY:
    print("⚠️ OPENAI_API_KEY not found in environment variables")
    print("Please set your OpenAI API key in your environment or .env file")

# Initialize clients
client = AsyncOpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="PLEASE_COPY_AND_PASTE_YOUR_DATABRICKS_MODEL_ENDPOINT"
)

openai_client = AsyncOpenAI(
    api_key=OPENAI_API_KEY,
)

print("✅ Clients initialized successfully")

✅ Clients initialized successfully


### Pydantic Models for Structured Output

In [3]:
class Format(BaseModel):
    answer: List[int]

## Functions

### Function: `load_evaluation_data`

Loads a JSONL file and returns a list of dictionaries.

- Opens the file and parses each line as JSON.  
- Handles missing files or parsing errors gracefully (prints error and returns `[]`).  
- Prints the number of items successfully loaded.

Useful for quickly reading evaluation datasets stored in JSONL format.

In [5]:
def load_evaluation_data(filepath: str) -> List[Dict]:
    """Load evaluation data from JSONL file"""
    data = []
    total_count = 0
    print(f"📁 Loading data from: {filepath}")
    
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                total_count += 1
                item = json.loads(line.strip())
                data.append(item)
    except FileNotFoundError:
        print(f"❌ File not found: {filepath}")
        return []
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return []
    
    print(f"✅ Total items loaded: {len(data)}")
    return data

### Function: `create_chunk_prompt_top_k`

This function builds a **prompt string** that asks a language model to pick and rank the most relevant text chunks for a given question.

**Key Steps**
1. **Determine Top-k Size**  
   - Uses `k` (default 10) if there are more than 10 chunks.  
   - Otherwise uses the total number of chunks.

2. **Compose Prompt**  
   - Starts with instructions to select and rank the `actual_k` most relevant chunks.  
   - Inserts the **question** and all provided text chunks, each labeled with its original index.

3. **Specify Output Format**  
   - Requests a final ordered list of chunk indices, e.g.  
     `[1st_most_relevant_index, 2nd_most_relevant_index, ..., kth_most_relevant_index]`.

**Purpose**  
This prompt can be sent to a large language model to **identify and rank the most relevant pieces of text** (chunks) before downstream tasks like question answering or summarization.

In [6]:
def create_chunk_prompt_top_k(question: str, chunks: List[str], chunk_indices: List[int], k: int = 10) -> str:
    """Ask model to select and rank only top-k most relevant chunks"""
    # Use k if chunks length > 10, else use chunks length
    actual_k = k if len(chunks) > 10 else len(chunks)
    
    prompt = f"""Identify the {actual_k} most relevant text chunks for answering this question, then rank them in order of relevance (best first).
Question: {question}
Text chunks:
"""
    for i, (chunk, orig_idx) in enumerate(zip(chunks, chunk_indices)):
        prompt += f"[Chunk Index {orig_idx}] {chunk}\n"
    prompt += f"""
Task: Select and rank the {actual_k} most relevant chunks among the given text chunks.
- Put the BEST chunk first
- Put the 2nd best chunk second  
- Continue until you have ranked your top {actual_k} chunks
Response Format: [1st_most_relevant_index, 2nd_most_relevant_index, ..., {actual_k}th_most_relevant_index]"""
   
    return prompt

### Function: `get_model_response`

Asynchronously gets a **financial-analysis response** from a specified model and extracts a ranked list.

- Prepends a **system prompt**: *"You are a helpful financial analyst."*  
- Sends combined messages to the target model (default `databricks-gpt-oss-120b`).  
- Extracts and parses the ranking list using an OpenAI helper (`gpt-4o-mini`).  
- Uses an `asyncio.Semaphore` to limit concurrent requests.  
- Returns an empty list on error.

Efficiently retrieves and structures ranking results from the model.

In [7]:
async def get_model_response(messages: List[Dict], model: str = "databricks-gpt-oss-120b", semaphore: asyncio.Semaphore = None) -> List[int]:
    """Get response from the model with financial analyst system prompt"""
    async with semaphore:
        system_message = {"role": "system", "content": "You are a helpful financial analyst."}
        
        full_messages = [system_message] + messages
        
        try:
            response = await client.chat.completions.create(
                messages=full_messages,
                model=model,
            )
            # Get the raw text response
            raw_output = response.choices[0].message.content[1]["text"].strip()
            
            # Use OpenAI to extract structured ranking
            extraction_response = await openai_client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": f"Extract the ranking list from this text. Return only the numbers in order as a list: {raw_output}"}
                ],
                response_format=Format,
            )
            
            return extraction_response.choices[0].message.parsed.answer
            
        except Exception as e:
            traceback.print_exc()
            print(f"❌ Error getting model response: {e}")
            # Return default ranking based on number of items expected
            return []

### Function: `extract_ranking_from_response`

Ensures the ranking list has the required length.

- If `response` has at least `num_items`, return the first `num_items`.
- Otherwise, pad with default indices (`0,1,2,...`) until the length reaches `num_items`.

Guarantees a fixed-size ranking list for downstream use.

In [8]:
def extract_ranking_from_response(response: List[int], num_items: int) -> List[int]:
    """Extract ranking list from model response"""
    # Ensure we have the right number of items
    if len(response) >= num_items:
        return response[:num_items]
    else:
        # Pad with default values if needed
        padded = response + list(range(len(response), num_items))
        return padded[:num_items]

### Function: `process_chunk_ranking_two_stage`

Handles **chunk ranking** for prompts with very high token counts using a two-stage strategy.

- **Token Check**  
  - Uses `tiktoken` to count tokens in the first message.  
  - If over 60,000 tokens, switches to multi-stage processing.

- **Multi-Stage Processing (if large)**  
  - Extracts `question` and `[Chunk Index N]` sections via regex.  
  - Splits chunks into three groups (first/second/third).  
  - For each group, calls `get_model_response` to find top candidates.  
  - Merges these top chunks and runs a final `get_model_response` to get the final ranking.

- **Single-Stage Processing (if normal)**  
  - Directly calls `get_model_response` and extracts top-10 ranking.

- **Error Handling**  
  - Logs parsing or runtime errors and returns an empty list on failure.

This approach efficiently **ranks large text sets** by first narrowing candidates in parallel and then re-ranking a smaller subset.

In [9]:
async def process_chunk_ranking_two_stage(messages: List[Dict], semaphore: asyncio.Semaphore, query_id: str) -> List[int]:
    """Process chunk ranking with multi-stage approach for high token count cases"""
    try:
        # Check if this is a high token case by examining the message content
        encoding = tiktoken.get_encoding("cl100k_base")
        content = messages[0].get('content', '')
        token_count = len(encoding.encode(content))
        
        if token_count > 60000:
            
            # Extract question and chunks from the message content
            # Find question
            question_start = content.find('Question:')
            question_end = content.find('\n', question_start)
            if question_start != -1 and question_end != -1:
                question = content[question_start + len('Question:'):question_end].strip()
            else:
                question = None
            
            # Find chunks using regex-like pattern matching
            chunks = []
            chunk_indices = []
            
            import re
            # Pattern to match [Chunk Index N] followed by content until next [Chunk Index] or Task:
            chunk_pattern = r'\[Chunk Index (\d+)\]\s*([\s\S]*?)(?=\[Chunk Index|Task:|$)'
            matches = re.findall(chunk_pattern, content)
            
            for i, match in enumerate(matches):
                orig_idx = int(match[0])
                chunk_content = match[1].strip()
                
                # Clean up chunk content - remove any task instructions that might be caught
                if 'Task:' in chunk_content:
                    chunk_content = chunk_content.split('Task:')[0].strip()
                
                if chunk_content:
                    chunks.append(chunk_content)
                    chunk_indices.append(orig_idx)
            
            if not question or not chunks:
                print("⚠️ Could not parse question and chunks, falling back to normal processing")
                response = await get_model_response(messages, semaphore=semaphore)
                predicted_ranking = extract_ranking_from_response(response, 10)
            else:
                # Split chunks into three parts
                third_point_1 = len(chunks) // 3
                third_point_2 = (len(chunks) * 2) // 3

                # First third
                first_third_chunks = chunks[:third_point_1]
                first_third_indices = chunk_indices[:third_point_1]
                first_prompt = create_chunk_prompt_top_k(question, first_third_chunks, first_third_indices, k=10)
                first_messages = [{"role": "user", "content": first_prompt}]
                first_response = await get_model_response(first_messages, semaphore=semaphore)
                first_top_3 = extract_ranking_from_response(first_response, 10)

                # Second third
                second_third_chunks = chunks[third_point_1:third_point_2]
                second_third_indices = chunk_indices[third_point_1:third_point_2]
                second_prompt = create_chunk_prompt_top_k(question, second_third_chunks, second_third_indices, k=10)
                second_messages = [{"role": "user", "content": second_prompt}]
                second_response = await get_model_response(second_messages, semaphore=semaphore)
                second_top_3 = extract_ranking_from_response(second_response, 10)

                # Third third
                third_third_chunks = chunks[third_point_2:]
                third_third_indices = chunk_indices[third_point_2:]
                third_prompt = create_chunk_prompt_top_k(question, third_third_chunks, third_third_indices, k=10)
                third_messages = [{"role": "user", "content": third_prompt}]
                third_response = await get_model_response(third_messages, semaphore=semaphore)
                third_top_4 = extract_ranking_from_response(third_response, 10)

                # Combine top results from each third
                combined_indices = first_top_3 + second_top_3 + third_top_4
                combined_chunks = []

                # Get chunks for the combined indices while preserving original indices
                for idx in combined_indices:
                    if idx in chunk_indices:
                        chunk_pos = chunk_indices.index(idx)
                        combined_chunks.append(chunks[chunk_pos])

                final_prompt = create_chunk_prompt_top_k(question, combined_chunks, combined_indices, k=10)
                final_messages = [{"role": "user", "content": final_prompt}]
                final_response = await get_model_response(final_messages, semaphore=semaphore)
                predicted_ranking = extract_ranking_from_response(final_response, 10)
                
        else:
            # Normal single-stage processing
            response = await get_model_response(messages, semaphore=semaphore)
            predicted_ranking = extract_ranking_from_response(response, 10)
        
        return predicted_ranking
    except Exception as e:
        traceback.print_exc()
        print(f"❌ Error processing chunk ranking item: {e}")
        return []

### Function: `process_single_item`

Processes one evaluation item to obtain a ranked list.

- Calls `get_model_response` with the given messages.  
- Uses `extract_ranking_from_response` to ensure the ranking has `num_items` elements.  
- Returns an empty list if any error occurs.

A simple wrapper for **single-item chunk ranking**.

In [10]:
async def process_single_item(messages: List[Dict], num_items: int, semaphore: asyncio.Semaphore) -> List[int]:
    """Process a single evaluation item"""
    try:
        # Get model response
        response = await get_model_response(messages, semaphore=semaphore)
        
        # Extract ranking from response
        predicted_ranking = extract_ranking_from_response(response, num_items)
        
        return predicted_ranking
        
    except Exception as e:
        print(f"❌ Error processing item: {e}")
        return []

### Functions: `evaluate_chunk_ranking` & `evaluate_document_ranking`

Run **end-to-end evaluation** for chunk and document ranking tasks.

#### `evaluate_chunk_ranking`
- Loads evaluation data and checks availability.
- For each item:
  - Uses `process_chunk_ranking_two_stage` to handle high-token cases.
  - Collects the top 5 ranked chunk indices for submission.
- Displays progress and summary (tasks completed, total submission entries).

#### `evaluate_document_ranking`
- Similar flow for document ranking, but:
  - Uses `process_single_item` (no multi-stage splitting).
  - Also records the top 5 ranked document indices.

Both functions return a **submission-ready list of dictionaries** with  
`sample_id` and `target_index` for each predicted top item.

In [11]:
async def evaluate_chunk_ranking(data_path: str, semaphore: asyncio.Semaphore) -> List[Dict]:
    """Evaluate chunk ranking task and return submission data"""
    print("\n🔍 CHUNK RANKING EVALUATION")
    print("="*50)
    data = load_evaluation_data(data_path)
    
    if not data:
        print("❌ No data loaded for chunk ranking")
        return []
    
    print(f"🎯 Evaluating {len(data)} chunk ranking items...")
    
    # Create tasks for concurrent processing
    tasks = []
    submission_data = []
    for idx, item in enumerate(data):
        messages = item['messages']
        query_id = item['_id']  # Use original _id from data
        # Use multi-stage chunk ranking process for high token cases
        task = process_chunk_ranking_two_stage(messages, semaphore, query_id)
        tasks.append((task, query_id))
    
    # Process all tasks with progress bar
    results_list = []
    for task_tuple in tqdm(tasks, desc="🔄 Processing chunk ranking", leave=False):
        task, query_id = task_tuple
        result = await task
        if result:
            results_list.append((result, query_id))
            # Add top 5 results to submission data
            for rank, doc_idx in enumerate(result[:5]):
                submission_data.append({'sample_id': query_id, 'target_index': doc_idx})
    
    print(f"✅ Completed {len(results_list)} chunk ranking tasks")
    print(f"📊 Generated {len(submission_data)} submission entries")
    return submission_data

In [12]:
async def evaluate_document_ranking(data_path: str, semaphore: asyncio.Semaphore) -> List[Dict]:
    """Evaluate document ranking task and return submission data"""
    print("\n📄 DOCUMENT RANKING EVALUATION")
    print("="*50)
    data = load_evaluation_data(data_path)
    
    if not data:
        print("❌ No data loaded for document ranking")
        return []
    
    print(f"🎯 Evaluating {len(data)} document ranking items...")
    
    # Create tasks for concurrent processing
    tasks = []
    submission_data = []
    for idx, item in enumerate(data):
        messages = item['messages']
        query_id = item['_id']  # Use original _id from data
        task = process_single_item(messages, 10, semaphore)
        tasks.append((task, query_id))
    
    # Process all tasks with progress bar
    results_list = []
    for task_tuple in tqdm(tasks, desc="🔄 Processing document ranking", leave=False):
        task, query_id = task_tuple
        result = await task
        if result:
            results_list.append((result, query_id))
            # Add top 5 results to submission data
            for rank, doc_idx in enumerate(result[:5]):
                submission_data.append({'sample_id': query_id, 'target_index': doc_idx})
    
    print(f"✅ Completed {len(results_list)} document ranking tasks")
    print(f"📊 Generated {len(submission_data)} submission entries")
    return submission_data

### Function: `save_submission_csv`

Saves prediction results to a **CSV file** in the required format.

- Creates a CSV with header: `sample_id, target_index`.  
- Writes each entry from `submission_data` as a row.  
- Prints the output file path and total entry count.  
- Logs an error message if file writing fails.

Convenient for generating **final submission files** from evaluation results.

In [13]:
def save_submission_csv(submission_data: List[Dict], filename: str):
    """Save submission data to CSV file in the required format"""
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['sample_id', 'target_index'])
            
            for entry in submission_data:
                writer.writerow([entry['sample_id'], entry['target_index']])
        
        print(f"💾 Submission file saved to {filename}")
        print(f"📊 Total entries: {len(submission_data)}")
    except Exception as e:
        print(f"❌ Error saving submission file: {e}")

## Data File Check

In [14]:
# Check if data files exist
chunk_ranking_path = "./output/chunk_ranking_kaggle_eval.jsonl"
document_ranking_path = "./output/document_ranking_kaggle_eval.jsonl"

print("🔍 Checking for required data files...")
print(f"📁 Chunk ranking file: {chunk_ranking_path}")
print(f"   Exists: {'✅' if os.path.exists(chunk_ranking_path) else '❌'}")
print(f"📁 Document ranking file: {document_ranking_path}")
print(f"   Exists: {'✅' if os.path.exists(document_ranking_path) else '❌'}")

if os.path.exists(chunk_ranking_path) and os.path.exists(document_ranking_path):
    print("\n🎉 All required files found! Ready to run evaluation.")
else:
    print("\n⚠️ Missing required data files. Please ensure both files exist in the ./output/ directory.")
    print("   You may need to run the data preparation script first.")

🔍 Checking for required data files...
📁 Chunk ranking file: ./output/chunk_ranking_kaggle_eval.jsonl
   Exists: ❌
📁 Document ranking file: ./output/document_ranking_kaggle_eval.jsonl
   Exists: ❌

⚠️ Missing required data files. Please ensure both files exist in the ./output/ directory.
   You may need to run the data preparation script first.


## Main Evaluation Pipeline

In [15]:
async def main():
    """Main evaluation function"""
    print("\n" + "="*60)
    print("🏆 KAGGLE RANKING EVALUATION PIPELINE")
    print("="*60)
    print("🤖 Model: databricks-gpt-oss-120b")
    print("👨‍💼 System Prompt: You are a helpful financial analyst.")
    print("📄 Output: CSV submission file only")
    print("🔄 Concurrency: 1 simultaneous request")
    print("="*60)
    
    # Create semaphore for limiting concurrent requests
    semaphore = asyncio.Semaphore(1)
    
    # Check files exist before starting
    if not os.path.exists(chunk_ranking_path) or not os.path.exists(document_ranking_path):
        print("❌ Required data files not found. Please check the file paths.")
        return
    
    # Evaluate chunk ranking and document ranking concurrently
    print("\n🚀 Starting evaluation...")
    chunk_task = evaluate_chunk_ranking(chunk_ranking_path, semaphore)
    doc_task = evaluate_document_ranking(document_ranking_path, semaphore)
    
    # Wait for both evaluations to complete
    print("\n⏳ Running both evaluations concurrently...")
    chunk_submission, doc_submission = await asyncio.gather(chunk_task, doc_task)
    
    # Combine submission data
    all_submission_data = chunk_submission + doc_submission
    
    # Save submission CSV
    save_submission_csv(all_submission_data, './kaggle_submission.csv')
    
    print("\n" + "="*60)
    print("🎊 EVALUATION COMPLETE!")
    print("="*60)
    print(f"🔍 Chunk ranking entries: {len(chunk_submission):,}")
    print(f"📄 Document ranking entries: {len(doc_submission):,}")
    print(f"📊 Total submission entries: {len(all_submission_data):,}")
    print(f"💾 Submission file: kaggle_submission.csv")
    print("\n🚀 Ready for Kaggle submission!")
    print("="*60)

## 🚀 Run the Complete Evaluation

Execute this cell to run the full evaluation pipeline. Make sure you have:

1. ✅ Set up your environment variables (DATABRICKS_TOKEN, OPENAI_API_KEY)
2. ✅ Have the evaluation data files in the `./output/` directory:
   - `chunk_ranking_kaggle_eval.jsonl`
   - `document_ranking_kaggle_eval.jsonl`
3. ✅ Installed all required packages

In [16]:
# Run the complete evaluation pipeline
await main()


🏆 KAGGLE RANKING EVALUATION PIPELINE
🤖 Model: databricks-gpt-oss-120b
👨‍💼 System Prompt: You are a helpful financial analyst.
📄 Output: CSV submission file only
🔄 Concurrency: 1 simultaneous request
❌ Required data files not found. Please check the file paths.


## 📊 View Results

Check the generated submission file and its contents:

In [17]:
import pandas as pd

# Load and display submission file if it exists
submission_file = './kaggle_submission.csv'
if os.path.exists(submission_file):
    df = pd.read_csv(submission_file)
    print(f"📊 Submission file shape: {df.shape}")
    print(f"\n📋 Sample data (first 10 rows):")
    print(df.head(10))
    print(f"\n🎯 Statistics:")
    print(f"   • Unique sample_ids: {df['sample_id'].nunique():,}")
    print(f"   • Sample ID range: {df['sample_id'].min()} to {df['sample_id'].max()}")
    print(f"   • Target index range: {df['target_index'].min()} to {df['target_index'].max()}")
    print(f"   • Total entries: {len(df):,}")
    
    # Show distribution of entries per sample_id
    entries_per_sample = df.groupby('sample_id').size()
    print(f"\n📈 Entries per sample_id distribution:")
    print(f"   • Mean: {entries_per_sample.mean():.1f}")
    print(f"   • Min: {entries_per_sample.min()}")
    print(f"   • Max: {entries_per_sample.max()}")
    print(f"   • Most common: {entries_per_sample.mode().iloc[0]} entries per sample")
    
else:
    print("❌ Submission file not found. Please run the evaluation first.")

❌ Submission file not found. Please run the evaluation first.


### 🚀 **Ready for Competition!**
The generated `kaggle_submission.csv` file can be directly uploaded to the Kaggle competition platform.

**Good luck! 🏆**